# Imports

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import torch
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

# Loading Dataset

In [5]:
train_df = pd.read_csv("twitter_training.csv", header = None)
test_df = pd.read_csv("twitter_validation.csv", header = None)

In [6]:
df = pd.concat([train_df, test_df])

In [7]:
df = df.drop([0, 1], axis="columns")

In [8]:
df.columns = ['Sentiment', 'Text']

In [9]:
df = df.dropna()

In [10]:
df = df.sample(1000)

In [11]:
df

,Sentiment,Text
13791,Negative,RhandlerR how does ur cover athletes jumpshot ...
36962,Neutral,New post: This Microsoft Office exploit was fi...
56844,Negative,@Ubisoft @Rainbow6Game Okay you tried with the...
63536,Negative,bruh @ EAMaddenNFL this h2h game's stuck on th...
70473,Neutral,"With the addition of AI Teammates, Auroa feels..."
...,...,...
14952,Neutral,"2.5K Bracket , Ranked after Turbo || !twitch !..."
44704,Negative,okay is the @Verizon network is ever going to ...
517,Neutral,.:: Ah yes. A very old picture of Demon Lottie...
56416,Positive,@Ubisoft @Rainbow6Game I would love that you b...


In [12]:
df['Sentiment'].unique()

array(['Negative', 'Neutral', 'Irrelevant', 'Positive'], dtype=object)

# Train Test Split

In [13]:
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

In [14]:
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# Converting to Hugging Face Dataset

In [15]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [16]:
train_dataset

Dataset({
    features: ['Sentiment', 'Text'],
    num_rows: 900
})

# Tokenization

In [17]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(example):
    return tokenizer(example['Text'], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_data, batched=True)
test_dataset = test_dataset.map(tokenize_data, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [18]:
label_dict = {'Positive': 0, 'Negative': 1, 'Neutral': 2, 'Irrelevant': 3}
def convert_sentiment_to_label(example):
    label = label_dict[example['Sentiment']]
    return {'labels': label}

train_dataset = train_dataset.map(convert_sentiment_to_label)
test_dataset = test_dataset.map(convert_sentiment_to_label)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

# Loading Model

In [55]:
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_dict))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Training

In [79]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [81]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,            # evaluation dataset
    compute_metrics=compute_metrics
)

In [82]:
trainer.train()

Step,Training Loss
10,1.262800
20,1.301900
30,1.233500
40,1.251400
50,1.313200
60,1.201300
70,1.165100
80,1.161200
90,1.117500
100,1.088900


TrainOutput(global_step=113, training_loss=1.2014482464410563, metrics={'train_runtime': 48.2454, 'train_samples_per_second': 18.655, 'train_steps_per_second': 2.342, 'total_flos': 236804202086400.0, 'train_loss': 1.2014482464410563, 'epoch': 1.0})

# Evaluation

In [85]:
trainer.evaluate()

{'eval_loss': 1.2190449237823486,
 'eval_accuracy': 0.49,
 'eval_f1': 0.4523195295270767,
 'eval_precision': 0.4230291005291005,
 'eval_recall': 0.49,
 'eval_runtime': 2.0754,
 'eval_samples_per_second': 48.183,
 'eval_steps_per_second': 3.373,
 'epoch': 1.0}

# Single Prediction

In [88]:
input_text = "I like this movie"
encoded_input = tokenizer(input_text, return_tensors='pt', truncation=True, max_length=512, padding='max_length').to('cuda')

model.eval()

with torch.no_grad():
    outputs = model(**encoded_input)

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predicted_index = predictions.argmax().item()

labels = ['Positive', 'Negative', 'Neutral', 'Irrelevant']
predicted_label = labels[predicted_index]
print(f"Predicted label: {predicted_label}")

Predicted label: Positive


# Saving Model

In [99]:
model.save_pretrained('model')
tokenizer.save_pretrained('model')

('model/tokenizer_config.json',
 'model/special_tokens_map.json',
 'model/vocab.txt',
 'model/added_tokens.json',
 'model/tokenizer.json')

In [19]:
model = AutoModelForSequenceClassification.from_pretrained('model')
tokenizer = AutoTokenizer.from_pretrained('model')